In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath('..'))
import django_initializer

In [36]:
from heros.configuration.models import ConfigDocument, Package, PackageVersion
from heros.configuration.serializers import ConfigDocumentSerializer
from git import Repo
from os import path
import os
import shutil
import json

document_types = ['context','object','method']
repo_base_path = path.join("..","repos")

class GitPackage:
    def __init__(self, package, version=None):
        if not package.remote:
            raise Exception("Remote repository not updated in package")
        self.package = package
        self.version = version
        self.repo_path = path.join(repo_base_path, package.code)
        self.docs_path = path.join(self.repo_path, "docs")
        if os.path.isdir(self.repo_path):
            shutil.rmtree(self.repo_path)
        self.repo = Repo.clone_from(package.remote,self.repo_path)

    def ensure_version(self):
        if not self.version:
            raise Exception("Version required")

    def checkout_version(self):
        self.ensure_version()
        if self.version.remote_commit == 'initial':
            self.repo.git.checkout('--orphan',self.version.version)
        else: 
            self.repo.git.checkout(self.version.version)

    def write_documents_to_repo(self):
        self.ensure_version()
        for document in self.version.config_documents.all():
            document_git_path = '/'.join([
                'docs', 
                document.document_type, 
                document.code + ".json"])

            document_path = path.join(
                self.repo_path ,
                'docs', 
                document.document_type, 
                document.code + ".json")

            data = ConfigDocumentSerializer(document).data
            os.makedirs(os.path.dirname(document_path), exist_ok=True)
            with open(document_path, "w") as outfile:
                json.dump(data, outfile,indent=4)
            self.repo.index.add(document_git_path)
    def commit_and_push(self, message):
        self.ensure_version()
        commit = self.repo.index.commit(message)
        self.repo.git.push('origin', version.version)
        version.local_commit = commit.hexsha
        version.remote_commit = commit.hexsha
        version.save()

    def update_version_status(self):
        

    def get_repo(self):
        return self.repo

version = PackageVersion.objects.get(pk=1)
repo = GitPackage(version.package,version)
repo.checkout_version()
repo.write_documents_to_repo()
repo.commit_and_push("Plublish at " + str(datetime.now()))



In [21]:
from heros.configuration.models import ConfigDocument, Package, PackageVersion
from heros.configuration.serializers import ConfigDocumentSerializer
from heros.configuration.git import GitPackage

version = PackageVersion.objects.get(pk=1)
repo = GitPackage(version.package,version)


# list(filter(os.path.isdir, os.listdir(repo.docs_path)))
# list(filter(os.path.isdir, os.path(repo.docs_path,)))

def get_directories(dir_path):
    directories = []
    for dir_item in os.listdir(dir_path):
        dir_item_path = os.path.join(dir_path, dir_item)
        if os.path.isdir(dir_item_path):
            directories.append(dir_item_path)
            directories.extend(get_directories(dir_item_path))
    return directories

dirs = get_directories(repo.docs_path)


['./repos/darvel.core/docs/context', './repos/darvel.core/docs/object']

In [13]:
import os

document_types = ['context','object','method']
repo_base_path = os.path.join(".","repos")
docs_path = os.path.join(repo_base_path, "docs")



FileNotFoundError: [Errno 2] No such file or directory: './repos/docs'